# OpenACC: 2X in 4 Steps (for C)

In this self-paced, hands-on lab, we will use [OpenACC](http://openacc.org/) directives to port a basic C program to an accelerator in four simple steps, achieving *at least* a two-fold speed-up.

Lab created by John Coombs, Mark Harris, and Mark Ebersole (Follow [@CUDAHamster](https://twitter.com/@cudahamster) on Twitter)

The following timer counts down to a five minute warning before the lab instance shuts down.  You should get a pop up at the five minute warning reminding you to save your work!

<script src="files/countdown_v5.0/countdown.js"></script>
<div id="clock" align="center"></div>
<script>
myDate = new Date();
curTime = Date.UTC(myDate.getUTCFullYear(), 
                   myDate.getUTCMonth(), 
                   myDate.getUTCDate(), 
                   myDate.getUTCHours(), 
                   myDate.getUTCMinutes(),
                   myDate.getUTCSeconds(),
                   myDate.getUTCMilliseconds());

function countdownComplete(){
  	alert("You only have five minutes left in the lab! Time to save your work - see the Post Lab section near the bottom.");
}
var myCD = new Countdown({
                         time  	: (1487297227753+110*60000-curTime)/1000,
                         target	 	: "clock",
                         onComplete	: countdownComplete,
                         rangeHi  : "minute",
                         hideLine	: true,
                         hideLabels	: false,
                         height	 	: 60,
                         width     : 150,
                         style     : "boring",
                    });
 </script>

---
Before we begin, let's verify [WebSockets](http://en.wikipedia.org/wiki/WebSocket) are working on your system.  To do this, execute the cell block *below* by giving it focus (clicking on it with your mouse), and hitting Ctrl-Enter, or by pressing the play button in the toolbar *above*.  If all goes well, you should see get some output returned below the grey cell.  If not, please consult the [Self-paced Lab Troubleshooting FAQ](https://developer.nvidia.com/self-paced-labs-faq#Troubleshooting) to debug the issue.

In [1]:
print "The answer should be three: " + str(1+2)

The answer should be three: 3


Next let's get information about the GPUs on the server by executing the cell below.

In [2]:
!nvidia-smi

Fri Feb 17 02:09:46 2017       
+------------------------------------------------------+                       
| NVIDIA-SMI 340.29     Driver Version: 340.29         |                       
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GRID K520           On   | 0000:00:03.0     Off |                  N/A |
| N/A   30C    P8    18W / 125W |     10MiB /  4095MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Compute processes:                                               GPU Memory |
|  GPU  

---
<p class="hint_trigger">If you have never before taken an IPython Notebook based self-paced lab from NVIDIA, click this green box.
      <div class="toggle_container"><div class="input_area box-flex1"><div class=\"highlight\">The following video will explain the infrastructure we are using for this self-paced lab, as well as give some tips on it's usage.  If you've never taken a lab on this system before, it's highly encourage you watch this short video first.<br><br>
<div align="center"><iframe width="640" height="390" src="http://www.youtube.com/embed/ZMrDaLSFqpY" frameborder="0" allowfullscreen></iframe></div>
<br>
<h2 style="text-align:center;color:red;">Attention Firefox Users</h2><div style="text-align:center; margin: 0px 25px 0px 25px;">There is a bug with Firefox related to setting focus in any text editors embedded in this lab. Even though the cursor may be blinking in the text editor, focus for the keyboard may not be there, and any keys you press may be applying to the previously selected cell.  To work around this issue, you'll need to first click in the margin of the browser window (where there are no cells) and then in the text editor.  Sorry for this inconvenience, we're working on getting this fixed.</div></div></div></div></p>

## Introduction to OpenACC

Open-specification OpenACC directives are a straightforward way to accelerate existing Fortran and C applications. With OpenACC directives, you provide hints via compiler directives (or 'pragmas') to tell the compiler where -- and how -- it should parallelize compute-intensive code for execution on an accelerator. 

If you've done parallel programming using OpenMP, OpenACC is very similar: using directives, applications can be parallelized *incrementally*, with little or no change to the Fortran, C or C++ source. Debugging and code maintenance are easier. OpenACC directives are designed for *portability* across operating systems, host CPUs, and accelerators. You can use OpenACC directives with GPU accelerated libraries, explicit parallel programming languages (e.g., CUDA), MPI, and OpenMP, *all in the same program.*

Watch the following short video introduction to OpenACC:

<div align="center"><iframe width="640" height="390" style="margin: 0 auto;" src="http://www.youtube.com/embed/c9WYCFEt_Uo" frameborder="0" allowfullscreen></iframe></div>

This hands-on lab walks you through a short sample of a scientific code, and demonstrates how you can employ OpenACC directives using a four-step process. You will make modifications to a simple C program, then compile and execute the newly enhanced code in each step. Along the way, hints and solution are provided, so you can check your work, or take a peek if you get lost.

If you are confused now, or at any point in this lab, you can consult the <a href="#FAQ">FAQ</a> located at the bottom of this page.

## The Value of 2X in 4 Steps

You can accelerate your applications using OpenACC directives and achieve *at least* a 2X speed-up, using 4 straightforward steps:

1. Characterize your application
2. Add compute directives
3. Minimize data movement
4. Optimize kernel scheduling

The content of these steps and their order will be familiar if you have ever done parallel programming on other platforms. Parallel programmers deal with the same issues whenever they tackle a new set of code, no matter what platform they are parallelizing an application for. These issues include:

+ optimizing and benchmarking the serial version of an application
+ profiling to identify the compute-intensive portions of the program that can be executed concurrently
+ expressing concurrency using a parallel programming notation (e.g., OpenACC directives)
+ compiling and benchmarking each new/parallel version of the application
+ locating problem areas and making improvements iteratively until the target level of performance is reached

The programming manual for some other parallel platform you've used may have suggested five steps, or fifteen. Whether you are an expert or new to parallel programming, we recommend that you walk through the four steps here as a good way to begin accelerating applications by at least 2X using OpenACC directives. We believe *being more knowledgeable about the four steps* will make the process of programming for an accelerator more understandable *and* more manageable. The 2X in 4 Steps process will help you use OpenACC on your own codes more productively, and get significantly better speed-ups in less time.

## Step 1 - Characterize Your Application

The most difficult part of accelerator programming begins before the first line of code is written. If your program is not highly parallel, an accelerator or coprocesor won't be much use. Understanding the code structure is crucial if you are going to *identify opportunities* and *successfully* parallelize a piece of code. The first step in OpenACC programming then is to *characterize the application*. This includes:

+ benchmarking the single-thread, CPU-only version of the application
+ understanding the program structure and how data is passed through the call tree
+ profiling the application and identifying computationally-intense "hot spots"
    + which loop nests dominate the runtime?
    + what are the minimum/average/maximum tripcounts through these loop nests?
    + are the loop nests suitable for an accelerator?
+ insuring that the algorithms you are considering for acceleration are *safely* parallel

Note: what we've just said may sound a little scary, so please note that as parallel programming methods go OpenACC is really pretty friendly: think of it as a sandbox you can play in. Because OpenACC directives are incremental, you can add one or two directives at a time and see how things work: the compiler provides a *lot* of feedback. The right software plus good tools plus educational experiences like this one should put you on the path to successfully accelerating your programs.

We will be accelerating a 2D-stencil called the Jacobi Iteration. Jacobi Iteration is a standard method for finding solutions to a system of linear equations. The basic concepts behind a Jacobi Iteration are described in the following video:

<div align="center"><iframe width="640" height="390" src="http://www.youtube.com/embed/UOSYi3oLlRs" frameborder="0" allowfullscreen></iframe></div>

Here is the serial C code for our Jacobi Iteration:

    #include <math.h>
    #include <string.h>
    #include <openacc.h>
    #include "timer.h"
    #include <stdio.h>

    #define NN 1024
    #define NM 1024

    float A[NN][NM];
    float Anew[NN][NM];

    int main(int argc, char** argv)
    {
        int i,j;
        const int n = NN;
        const int m = NM;
        const int iter_max = 1000;
        const double tol = 1.0e-6;
        double error     = 1.0;
    
        memset(A, 0, n * m * sizeof(float));
        memset(Anew, 0, n * m * sizeof(float));
        
        for (j = 0; j < n; j++)
        {
            A[j][0]    = 1.0;
            Anew[j][0] = 1.0;
        }
    
        printf("Jacobi relaxation Calculation: %d x %d mesh\n", n, m);
    
        StartTimer();
        int iter = 0;
    
        while ( error > tol && iter < iter_max )
        {
            error = 0.0;

            for( j = 1; j < n-1; j++)
            {
                for( i = 1; i < m-1; i++ )
                {
                    Anew[j][i] = 0.25 * ( A[j][i+1] + A[j][i-1]
                                    + A[j-1][i] + A[j+1][i]);
                    error = fmax( error, fabs(Anew[j][i] - A[j][i]));
                }
            }
        
            for( j = 1; j < n-1; j++)
            {
                for( i = 1; i < m-1; i++ )
                {
                     A[j][i] = Anew[j][i];    
                }
            }

                if(iter % 100 == 0) printf("%5d, %0.6f\n", iter, error);
        
            iter++;
        }

        double runtime = GetTimer();
 
        printf(" total: %f s\n", runtime / 1000);
    }

In this code, the outer 'while' loop iterates until the solution has converged, by comparing the computed error to a specified error tolerance, *tol*. The first of two sets of inner nested loops applies a 2D Laplace operator at each element of a 2D grid, while the second set copies the output back to the input for the next iteration.

###Benchmarking

Before you start modifying code and adding OpenACC directives, you should benchmark the serial version of the program. To facilitate benchmarking after this and every other step in our parallel porting effort, we have built a timing routine around the main structure of our program -- a process we recommend you follow in your own efforts. Let's run the `task1.c` file without making any changes -- using the *-fast* set of compiler options on the serial version of the Jacobi Iteration program -- and see how fast the serial program executes. This will establish a baseline for future comparisons.  Execute the following two cells to compile and run the program.

In [ ]:
# To be sure we see some output from the compiler, we'll echo out "Compiled Successfully!" 
#(if the compile does not return an error)
!pgcc -fast -o task1_pre_out task1/task1.c && echo "Compiled Successfully!"

In [ ]:
# Execute our single-thread CPU-only Jacobi Iteration to get timing information.  Make sure you compiled successfully in the 
# above cell first.
!./task1_pre_out

### Quality Checking/Keeping a Record

This is a good time to briefly talk about having a quality check in your code before starting to offload computation to an accelerator (or do any optimizations, for that matter).  It doesn't do you any good to make an application run faster if it does not return the correct results.  It is thus very important to have a quality check built into your application before you start accelerating or optimizing.  This can be a simple value print out (one you can compare to a non-accelerated version of the algorithm) or something else.

In our case, on every 100th iteration of the outer `while` loop, we print the current max error. (You just saw an example when we executed *task1_pre_out*.) As we add directives to accelerate our code later in this lab, you can look back at these values to verify that we're getting the correct answer.  These print-outs also help us verify that we are converging on a solution -- which means that we should see, as we proceed, that the values are approaching zero.

**Note:** NVIDIA GPUs implement IEEE-754 compliant floating point arithmetic just like most modern CPUs. However, because floating point arithmetic is not associative, the order of operations can affect the rounding error inherent with floating-point operations: you may not get exactly the same answer when you move to a different processor. Therefore, you'll want to make sure to verify your answer within an acceptable error bound. Please read [this](https://developer.nvidia.com/content/precision-performance-floating-point-and-ieee-754-compliance-nvidia-gpus) article at a later time, if you would like more details.

*After each step*, we will record the results from our benchmarking and correctness tests in a table like this one: 

|Step| Execution       | ExecutionTime (s)     | Speedup vs. 1 CPU Thread       | Correct? | Programming Time |
|:--:| --------------- | ---------------------:| ------------------------------:|:--------:| -----------------|
|1   | CPU 1 thread    | 3.130             |                                | Yes      |                |  |

*Note: Problem Size: 1024 x 1024; System Information: GK104; Compiler: PGI 14.3*

(The execution times quoted will be times we got running on our GK104 -- your times throughout the lab may vary for one reason or another.)

You may also want to track how much time you spend porting your application, step by step, so a column has been included for recording time spent.

### Profiling

Back to our lab. Your objective in the step after this one (Step 2) will be to modify `task2.c` in a way that moves the most computationally intensive, independent loops to the accelerator. With a simple code, you can identify which loops are candidates for acceleration with a little bit of code inspection. On more complex codes, a great way to find these computationally intense areas is to use a profiler (such as PGI's PGPROF or open-source *gprof*) to determine which functions are consuming the largest amounts of compute time. To profile a C program on your own workstation, you'd type the lines below on the command line, but in this workshop, you just need to execute the following cell, and then click on the link below it to see the pgprof interface

In [ ]:
%%bash
pgcc -Minfo=ccff -o task1/task1_simple_out task1/task1_simple.c
pgcollect task1/task1_simple_out

In this lab, to open the PGI profiler in a new window <a href="/vnc" onclick="window.open(this.href, 'Profiler',
'left=20,top=20,width=1028,height=772,toolbar=1,resizable=0'); return false;">click here</a>.

As in real life, in this lab you may need to tell pgprof where various files are in your directory structure.

<div align="center"><img src="files/profiling_parameters_c.png" width="60%"></div>

If so, click on *file*, then *open profile*. Use the browser buttons to navigate to *pgprof.out* and *task1_simple_out.exe* in the folder task1, then edit the source to match. Close the window by pressing OK. The profiler window should now look like this:

<div align="center"><img src="files/profiler_window_c.png" width="60%"></div>

Clicking on *main* in the pgprof.out pane will show you how much time your system is spending executing each line of task1.c:

<div align="center"><img src="files/profiler_task1_c.png" width="60%"></div>

In our Jacobi code sample, you can see that lines 42, 45 and 46 in loop one, and line 54 in loop two account for nearly all of our execution time. The compute-intensive part of our code is the two for-loops nested inside the while loop in the function *main*. (The profiler is a pretty useful tool!)

Let's see what it takes to accelerate those loops.

## Step 2 - Add Compute Directives

In C, an OpenACC directive is indicated in the code by '#pragma acc *your directive*'. This is very similar to OpenMP programming and gives hints to the compiler on how to handle the compilation of your source. If you are using a compiler which does not support OpenACC directives, it will simply ignore the '#pragma acc' directives and move on with the compilation.

In Step 2, you will add compute regions around your expensive parallel loop(s). The first OpenACC directive you're going to learn about is the *kernels* directive. The kernels directive gives the compiler a lot of freedom in how it tries to accelerate your code - it basically says, "Compiler, I believe the code in the following region is parallelizable, so I want you to try and accelerate it as best you can."

Like most OpenACC directives in C/C++, the kernels directive applies to the structured code block immediately following the #pragma acc *directive*. For example, each of the following code samples instructs the compiler to generate a kernel -- from suitable loops -- for execution on an accelerator:

    #pragma acc kernels
    {
        // accelerate suitable loops here  
    }
    // but not these loops

or

    #pragma acc kernels
    for ( int i = 0; i < n; ++i ) 
      {  // body of for-loop
        ... // The for-loop is a structured block, so this code will be accelerated
      }
    ... // Any code here will not be accelerated since it is outside of the for-loop

One, two or several loops may be inside the structured block, the kernels directive will try to parallelize it, telling you what it found and generating as many kernels as it thinks it safely can. At some point, you will encounter the OpenACC *parallel* directive, which provides another method for defining compute regions in OpenACC. For now, let's drop in a simple OpenACC `kernels` directive in front of and embracing *both* the two for-loop codeblocks that follow the while loop. The kernels directive is designed to find the parallel acceleration opportunities implicit in the for-loops in the Jacobi Iteration code.  

To get some hints about how and where to place your kernels directives, click on the green boxes below.  When you feel you are done, **make sure to save the task2.c file you've modified with File -> Save, and continue on.**  If you get completely stuck, you can look at task2_solution.c to see the answer.

<p class="hint_trigger">Hint #1
      <div class="toggle_container"><div class="input_area box-flex1"><div class=\"highlight\">Remember that in C, an OpenACC directive applies to the next structured code block.  So for example, the following applies the <code>kernels</code> directive to the outer <code>for</code> loop and everything inside of it:<pre><code>  #pragma acc kernels
    for ( int i = 0; i < n-1; i++ )
    {
        for ( int j = 0; j < n-1; j++)
            ...
    }</code></pre></div></div></div></p>
<p class="hint_trigger">Hint #2
      <div class="toggle_container"><div class="input_area box-flex1"><div class=\"highlight\">If you choose to use only one `#pragma acc kernels` region -- which we recommend, because it demonstrates the power of the *kernels* directive -- you will need to add some additional `{ }` brackets so it applies to the correct region of code.  </div></div></div></p>

<iframe id="task2" src="task2" width="100%" height="500px">
  <p>Your browser does not support iframes.</p>
</iframe>

Let's now compile our task2.c file by executing the cell below with Ctrl-Enter (or press the play button in the toolbar above).

In [ ]:
# Compile the task2.c file with the pgcc compiler
# -acc tells the compiler to process the source recognizing #pragma acc directives
# -Minfo tells the compiler to share information about the compilation process
!pgcc -acc -Minfo -o task2_out task2/task2.c && echo "Compiled Successfully"

If you successfully added `#pragma acc kernels` in the proper spots, you should see the following in the output of the compiler:

    main:
         36, Generating present_or_copyin(Anew[1:1022][1:1022])
             Generating copyin(A[:][:])
             Generating copyout(A[1:1022][1:1022])
             Generating NVIDIA code
         41, Loop is parallelizable
         43, Loop is parallelizable
             Accelerator kernel generated
             41, #pragma acc loop gang /* blockIdx.y */
             43, #pragma acc loop gang, vector(128) /* blockIdx.x threadIdx.x */
             47, Max reduction generated for error
         52, Loop is parallelizable
         54, Loop is parallelizable
             Accelerator kernel generated
             52, #pragma acc loop gang /* blockIdx.y */
             54, #pragma acc loop gang, vector(128) /* blockIdx.x threadIdx.x */
             Memory copy idiom, loop replaced by call to __c_mcopy4
             
If you do not get similar output, please check your work and try re-compiling.  If you're stuck, you can compare what you have to task2_solution.c in the editor above.

*The output provided by the compiler is extremely useful, and should not be ignored when accelerating your own code with OpenACC.*  Let's break it down a bit and see what it's telling us.

1. First since we used the `-Minfo` command-line option, we will see all output from the compiler.  If we were to use `-Minfo=accel` we would only see the output corresponding to the accelerator, in this case an NVIDIA GPU.
2. The first line of the output, *main*, tells us which function the following information is in reference to.
3. The line starting with `41, Loop is parallelizable` of the output tells us that on line `41` in our source, an accelerated kernel was generated.  This is the the loop just after where we put our `#pragma acc kernels`.
4. The following lines provide more details on the accelerator kernel on line 42.  It shows we created a parallel OpenACC `loop`. This loop is made up of gangs (a grid of blocks in CUDA language) and vector parallelism (threads in CUDA language) with the vector size being 128 per gang.
5. At line 54, the compiler tells us it found another loop to accelerate.
6. The rest of the information concerns data movement which we'll get into later in this lab.

So as you can see, lots of useful information is provided by the compiler, and it's very important that you carefuly inspect this information to make sure the compiler is doing what you've asked of it.

Finally, let's execute this program to verify we are getting the correct answer (execute the cell below). 

Once you feel your code is correct, try running it by executing the cell block below.  You'll want to review our quality check from the beginning of task2 to make sure you didn't break the functionality of your application.

In [ ]:
!./task2_out

Let's record our results in the table:

|Step| Execution    | Time(s)     | Speedup vs. 1 CPU Thread  | Correct? | Programming Time |
| -- || ------------ | ----------- | ------------------------- | -------- | ---------------- |
|1| CPU 1 thread |3.139      |                           |          | |
|2| Add kernels directive  |4.697      | 0.67X                    | Yes      | ||

*Note: Problem Size: 1024x1024; System Information: GK104; Compiler: PGI 14.3*


Now, if your solution is similar to the one in task2_solution.c, you have probably noticed that we're executing **slower** than the non-accelerated, CPU-only version we started with.  What gives?!

The compiler feedback we collected earlier tells you quite a bit about data movement, and you can collect even more by setting an environment variable (**export PGI_ACC_TIME=1**) and then running the compiled code. (We'll use this later in Step 4.) The reason for our slowdown in this step is *excessive data movement*: both regions spent the majority of their time *copying data*. (As in all modern computing, the movement of data is where applications spend most of their time and the host's computing power.)  This problem of copying back and forth on every iteration of a loop is sometimes called "data sloshing".

The OpenACC compiler can only work with the information we have given it.  It knows we need the `A` and `Anew` arrays on the GPU for each of our two accelerated sections, but we didn't tell it anything about what happens to the data outside of those sections.  Without this knowledge, it has to copy the full arrays *to the GPU and back to the CPU* for each accelerated section, *every time* it went through the while loop.  That is a LOT of wasted data transfers.

Ideally, we would just transfer `A` and `Anew` to the GPU at the beginning of the Jacobi Iteration, and then only transfer `A` back to the CPU at the end.

Because overall accelerator performance is detetermined largely by how well memory transfers are optimized, the OpenACC specification defines the `data` directive and several modifying clauses to manage all the various forms of data movement.


## Step 3 - Manage Data Movement

We need to give the compiler more information about how to reduce unnecessary data movement for the Jacobi Iteration.  We are going to do this with the OpenACC `data` directive and some modifying clauses defined in the OpenACC specification. 

In C, the `data` directive applies to the next structured code block.  The compiler will manage data according to the provided clauses.  It does this at the beginning of the `data` directive code block, and then again at the end.  Some of the clauses available for use with the `data` directive are:

* `copy( list )` - Allocates memory on GPU and copies data from host to GPU when entering region and copies data to the host when exiting region.
* `copyin( list )` - Allocates memory on GPU and copies data from host to GPU when entering region.
* `copyout( list )` - Allocates memory on GPU and copies data to the host when exiting region.
* `create( list )` - Allocates memory on GPU but does not copy.
* `present( list )` - Data is already present on GPU from another containing data region.

As an example, the following directive copies array A to the GPU at the beginning of the code block, and back to the CPU at the end.  It also copies arrays B and C *to the CPU* at the *end* of the code block, but does **not** copy them both to the GPU at the beginning:

<pre><code>#pragma acc data copy( A ), copyout( B, C )
{
   ....
}</code></pre>

For detailed information on the `data` directive clauses, you can refer to the OpenACC [1.0](http://www.openacc.org/sites/default/files/OpenACC.1.0_0.pdf) or [2.0](http://www.openacc.org/sites/default/files/OpenACC%202%200.pdf) specifications.

In the text editor below, see if you can add in a `data` directive to minimize data transfers in the Jacobi Iteration. There's a place for  the `create` clause in this exercise too.   As usual, there are some hints provided, and you can look at task3_solution.c to see the answer if you get stuck or want to check your work.  **Don't forget to save with File -> Save in the editor below before moving on.**

<p class="hint_trigger">Hint #1
      <div class="toggle_container"><div class="input_area box-flex1"><div class=\"highlight\">You should only have to worry about managing the transfer of data in arrays <code>A</code> and <code>Anew</code>.</div></div></div></p>
<p class="hint_trigger">Hint #2
      <div class="toggle_container"><div class="input_area box-flex1"><div class=\"highlight\">You want to put the data directive just above the outer <code>while</code> loop.</div></div></div></p>
<p class="hint_trigger">Hint #3
      <div class="toggle_container"><div class="input_area box-flex1"><div class=\"highlight\">You'll want to `copy( A )` so it is transferred to the GPU and back again after the final iterations through the `data` region.  But you only need to `create( Anew )` as it is just used for temporary storage on the GPU, so there is no need to ever transfer it back and forth.</div></div></div></p>      

<iframe id="task3" src="task3" width="100%" height="500px">
  <p>Your browser does not support iframes.</p>
</iframe>

Once you think you have task3.c saved with a directive to manage data transfer, compile it with the below cell and note the changes in the compiler output in the areas discussing data movement (lines starting with `Generating ...`). Then modify Anew using the `create` clause, if you haven't yet, and compile again.

In [ ]:
!pgcc -fast -acc -Minfo=accel -o task3_out task3/task3.c && echo "Compiled Successfully"

How are we doing on our timings? Let's execute our step 3 program and see if we have indeed accelerated the application versus the execution time we recorded after Step #2.

In [ ]:
!./task3_out

After making these changes, our accelerator code is much faster -- with just a few lines of OpenACC directives we have made our code more than twice as fast by running it on an accelerator, as shown in this table.

|Step| Execution                 | Time (s)               | Speedup vs. 1 CPU thread | Correct? | Programming Time |
| -- | ------------------------- | ---------------------- | ------------------------ | -------- | ---------------- |
|1| CPU 1 thread              | 3.130                |                          |          | |
|2| Add kernels directive     | 4.697                  |  0.67X                   |          | |
|3| Manage data movement        | 0.628                  |  4.98X                  | Yes      | ||

*Note: Problem Size: 1024x1024; System Information: GK104; Compiler: PGI 14.3*

We are making good progress, but we can still improve performance.

## Step 4 - Optimize Kernel Scheduling

The final step in our tuning process is to tune the OpenACC compute region schedules using the *gang* and *vector* clauses. These clauses let us use OpenACC directives to take more explicit control over how the compiler parallelizes our code *for the accelerator we will be using*. 

Kernel scheduling optimizations *may* give you significantly higher speedup, but be aware that these particular optimizations can significantly reduce performance portability. The vast majority of the time, the default kernel schedules chosen by the OpenACC compilers are quite good, but other times the compiler doesn't do as well. Let's spend a little time examining how we could do better, if we were in a situation where we felt we needed to. 

First, we need to get some additional insight into how our Jacobi Iteration code with the data optimizations is running on the accelerator. Let's run the C code with all your data movement optimizations on the accelerator again, this time setting the environment variable PGI_ACC_TIME that we mentioned in Step 2.

In [ ]:
%%bash
export PGI_ACC_TIME=1
pgcc -acc -Minfo=accel -o accel_timing_out task3/task3.c
./accel_timing_out

This generates some information we haven't seen previously from the PGI compiler:

    Accelerator Kernel Timing data
    /home/gpudev1/notebook/task3/task3_solution.c
      main  NVIDIA  devicenum=0
        time(us): 391,494
        34: data region reached 1 time
            34: data copyin reached 1 time
                 device time(us): total=479 max=479 min=479 avg=479
            68: data copyout reached 1 time
                 device time(us): total=519 max=519 min=519 avg=519
        37: compute region reached 1000 times
            44: kernel launched 1000 times
                grid: [8x1022]  block: [128]
                 device time(us): total=248,508 max=268 min=242 avg=248
                elapsed time(us): total=258,569 max=453 min=255 avg=258
            44: reduction kernel launched 1000 times
                grid: [1]  block: [256]
                 device time(us): total=27,531 max=83 min=25 avg=27
                elapsed time(us): total=37,589 max=93 min=35 avg=37
            55: kernel launched 1000 times
                grid: [8x1022]  block: [128]
                 device time(us): total=114,457 max=170 min=110 avg=114
                elapsed time(us): total=124,880 max=303 min=121 avg=124

There is a lot of information here about how the compiler mapped the computational kernels in our program to our particular accelerator (in this case, an NVIDIA GPU). We can see three regions. The first one is the memcopy loop nest starting on line 34, which takes only a tiny fraction of the 0.39 seconds of total system time. The second region is the nested computation loop starting on line 44, which takes about 0.25 seconds. The copyback (*copyout*) loop then executes beginning with line 68. We can see that region takes very little time -- which tells us there is no other part of the program that takes significant time. If we look at the main loop nests, we can see these lines: 

The terms *grid* and *block* come from the CUDA programming model. A GPU executes groups of threads called *thread blocks*. To execute a kernel, the application launches a *grid* of these thread blocks. Each block runs on one of the GPUs *multiprocessors* and is assigned a certain range of IDs that it uses to address a unique data range. In this case our thread blocks have 128 threads each. The grid the compiler has constructed is also 2D, 8 blocks wide and 1022 blocks tall. This is just enough to cover our 1024x1024 grid. But we don't really need that many blocks -- if we tell the compiler to launch fewer, it will automatically generate a sequential loop over data blocks within the kernel code run by each thread.

*Note: You can let the compiler do the hard work of mapping loop nests, unless you are certain you can do it better (and portability is not a concern.) When you decide to intervene, think about different parallelization strategies (loop schedules): in nested loops, distributing the work of the outer loops to the GPU multiprocessors (on PGI = gangs) in 1D grids. Similarly, think about mapping the work of the inner loops to the cores of the multiprocessors (CUDA threads, vectors)  in 1D blocks. The grids (gangs) and block (vector) sizes can be viewed by setting the environment variable ACC_NOTIFY. To get you started, here are some experiments we conducted for these computational kernels and this accelerator:*

| Accelerator | Grid          |Outer Loop Gang | Outer Loop Vector | Inner Loop Gang | Inner Loop Vector | Seconds |
| ----------- | ------------- |------------- | --------------- | ------------- | --------------- | ------- |
| GK104       | 1024x1024     |              |               8 |               |              32 | 0.516   |
|             |               |              |               4 |               |              64 | 0.516   |
|             |               |              |                 |            8  |              32 | 0.516   |
|             |               |              |                 |           16  |              32 | 0.585  |
|             |               |              |                 |            4  |              64 | 0.676   |

Try to modify the code for the main computational loop nests in the window below. You'll be using the openacc loop constructs `gang()` and  `vector()`. Look at task4_solution.c if you get stuck:


<p class="hint_trigger">Hint #1
      <div class="toggle_container"><div class="input_area box-flex1"><div class=\"highlight\">You'll want a gang() and vector() clause on the inner loops, but you may want to let the compiler decide the dimensions of the outer loops.In that case, you can use a loop directive without any modifying clauses.</div></div></div></p>

<iframe id="task4" src="task4" width="100%" height="500px">
  <p>Your browser does not support iframes.</p>
</iframe>

After you've made some changes, save your work, then compile and run in the boxes below:

In [ ]:
!pgcc -acc -Minfo=accel -o task4_out task4/task4.c && echo "Compiled Successfully"

In [ ]:
!./task4_out

Looking at task4_solution.c, the gang(8) clause on the inner loop tells it to launch 8 blocks in the X(column) direction. The vector(32) clause on the inner loop tells the compiler to use blocks that are 32 threads (one warp) wide. The absence of clause on the outer loop lets the compiler decide how many rows of threads and how many blocks to use in the Y(row) direction. We can see what it says, again, with:

In [ ]:
%%bash
export PGI_ACC_TIME=1
./task4_out

*Note: we usually want the inner loop to be vectorized, because it allows coalesced loading of data from global memory. This is almost guaranteed to give a big performance increase. Other optimizations are often trial and error. When selecting grid sizes, the most obvious mapping is to have*

    the number of gangs * the number of workers * the number of vectors = the total problem size. 
    
*We may choose to manipulate this number, as we are doing here, so that each thread does multiple pieces of work -- this helps amortize the cost of setup for simple kernels.*

*Note: Low-level languages like CUDA C/C++ offer more direct control of the hardware. You can consider optimizing your most critical loops in CUDA C/C++ if you need to extract every last bit of performance from your application, while recognizing that doing so may impact the portability of your code. OpenACC and CUDA C/C++ are fully interoperable.*

A similar change to the copy loop nest benefits performance by a small amount. After you've made all your changes (look at task4_solution.c to be sure) compile your code below:

In [ ]:
!pgcc -acc -Minfo=accel -o task4_out task4/task4.c  & echo "Compiled Successfully"

Then run it and record the run time of the optimized code in the table:

In [ ]:
!./task4_out

Here is the perfomance after these final optimizations:

|Step| Execution            | Time (s)      | Speedup vs. 1 CPU Thread        | Correct? | Programming Time |
| -- | -------------------- | ------------- | ------------------------------- | -------- | ---------------- |
|1| CPU 1 thread         | 3.130       |                                 |          | |
|2| Add kernels directive     | 4.697                  |  0.67X                   |          | |
|3| Manage data movement        | 0.628                  |  4.98X                  |           | |
|4| Optimize kernel scheduling | 0.518          | 6.04X                       | Yes      | ||

*Note: Problem Size: 1024x1024; System Information: GK104; Compiler: PGI 14.3*


At this point, some of you may be wondering what kind of speed-up we get against the OpenMP version of this code.  If you look at task3_omp.c in the text editor above, you can see a simple OpenMP version of the Jacobi Iteration code.  Running this using 8-OpenMP threads on an Intel Xeon x5570, our Kepler GK104 is just slightly slower.  The reason for this is that in this lab so far we've been using a rather small matrix in our example: 1024x1024.  This size was chosen in the interest of keeping the time you are spending here reasonable, as larger matrices take longer to run, especially before we optimize for data movement.

If we scale the matrix up to a more realistic size, say 4096x4096, our Kepler GK104 GPU becomes significantly faster than the 8-OpenMP thread version.  If you have some time remaining in this lab, feel free to compile & run the OpenMP and OpenACC versions below with the larger matrices.

First, compile the OpenMP version, with two threads on each of the 4 cores of the SandyBridge CPUs:

In [ ]:
%%bash
export OMP_NUM_THREADS=8
pgcc -fast -mp -Minfo -o task4_4096_omp task4/task4_4096_omp.c

Now run the OpenMP code you just created, and record your results in the new table for the larger matrix.

*Note: because our dataset has now grown by 16-fold your CPU may not seem as responsive. We're using `-Minfo` in the compile so you can see that something is indeed happening, but you may need to be patient*.

In [ ]:
!./task4_4096_omp

Now, compile and run the OpenACC solution for the larger 4096x4096 matrix using the next two boxes:

In [ ]:
!pgcc -acc -Minfo=accel -o task4_4096_out task4/task4_4096_solution.c && echo "Compiled Successfully"

In [ ]:
!./task4_4096_out

Here's our comparison with the larger matrix size:

| Execution            | matrix size | Time (s) | Speedup vs. 8 CPU threads | Correct? | Programming Time |
| -------------------- | ----------- | -------- | ------------------------- |          | |
| CPU 8   threads      | 4096x4096       | 55.504     |                           |          | |
| GPU optimized kernel | 4096x4096        | 3.605      | 15.40X                       |   Yes    | ||

*Note: System Information: GK104; Compiler: PGI 14.3*

## Learn More

If you are interested in learning more about OpenACC, you can use the following resources:

* [openacc.org](http://openacc.org/)
* [OpenACC on CUDA Zone](https://developer.nvidia.com/openacc)
* Search or ask questions on [Stackoverflow](http://stackoverflow.com/questions/tagged/openacc) using the openacc tag
* Get a [30-day trial](http://www.nvidia.com/object/openacc-gpu-directives.html) of the PGI OpenACC compiler *and* visit the PGI support forums at www.pgroup.com/userforum
* Attend an in-depth workshop offered by XSEDE (https://portal.xsede.org/overview) or a commercial provider (see the 'education' page at OpenACC.org)

---

<a id="post-lab"></a>
## Post-Lab

Finally, don't forget to save your work from this lab before time runs out and the instance shuts down!!

1. Save this IPython Notebook by going to `File -> Download as -> IPython (.ipynb)` at the top of this window
2. You can execute the following cell block to create a zip-file of the files you've been working on, and download it with the link below.

In [ ]:
%%bash
rm -f openacc_files.zip
zip -r openacc_files.zip task*/*.c task*/*.h

**After** executing the above zip command, you should be able to download the zip file [here](files/openacc_files.zip)

<a id="FAQ"></a>
---
# Lab FAQ

Q: I'm encountering issues executing the cells, or other technical problems?<br>
A: Please see [this](https://developer.nvidia.com/self-paced-labs-faq#Troubleshooting) infrastructure FAQ.

<style>
p.hint_trigger{
  margin-bottom:7px;
  margin-top:-5px;
  background:#64E84D;
}
.toggle_container{
  margin-bottom:0px;
}
.toggle_container p{
  margin:2px;
}
.toggle_container{
  background:#f0f0f0;
  clear: both;
  font-size:100%;
}
</style>
<script>
$("p.hint_trigger").click(function(){
   $(this).toggleClass("active").next().slideToggle("normal");
});
   
$(".toggle_container").hide();
</script>